In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gpt-4o-mini","api_key":OPENAI_API_KEY}

# Task

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''

In [3]:
# creating a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task,"role": "user"}])

In [6]:
reply

'**Harnessing Hope: The Future of AI**\n\nHope AI is revolutionizing the way we interact with technology by infusing optimism into artificial intelligence. This innovative approach focuses on enhancing human potential, empowering individuals to leverage AI tools for personal and societal growth. By prioritizing ethical considerations and fostering creativity, Hope AI aims to create solutions that uplift and inspire. As we embrace this forward-thinking mindset, we open doors to groundbreaking advancements in mental health support, education, and beyond. Let’s harness the power of hope and transform our relationship with AI into one of collaboration and positivity. The future looks bright!'

In [7]:
# Create a critic agent to reflect on the work of the writer agent.

In [8]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of the writer and only provide constructive feedback to help improve the quality of the content.",
)

In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg")

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: Pioneering a Brighter Tomorrow** 

In a world often clouded by uncertainty, Hope AI emerges as a beacon of optimism. This innovative technology harnesses artificial intelligence to address pressing global challenges, from climate change to healthcare disparities. By leveraging data-driven insights, Hope AI empowers communities to make informed decisions and foster resilience. Its user-friendly platform enables individuals and organizations alike to collaborate on solutions that uplift society. As we navigate an increasingly complex landscape, Hope AI stands at the forefront, reminding us that technology can spark positive change and inspire hope for a better future.

-------------------------------------------------------------

# Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",)

In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [14]:
# Orchestrate the nested chats to solve the task

recipient: The agent that is about to receive the message (e.g., SEO_reviewer).

messages: This is the most important part. It is a list of all messages exchanged in the current session between the two agents that triggered the nested chat (in your case, the critic and the writer).

sender: The agent that is initiating the nested sequence (the critic).

config: Any additional configuration passed through llm_config.

In [15]:
#This function is called inside the conversation and it takes in the writer's response as the input message to give its review

def reflection_message(recipient, messages, sender, config):
    content = recipient.chat_messages_for_summary(sender)[-1]['content']
    return f"Review the following content:\n\n {content}"

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    
    {
     "recipient": legal_reviewer,
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    
    {
     "recipient": ethics_reviewer,
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(review_chats,trigger=writer)
# The trigger is being set like whenever the critic gets a message from writer the nested chat sequence should be triggered

In [17]:
res = critic.initiate_chat(
                           recipient=writer,
                           message=task,
                           max_turns=2,
                           summary_method="last_msg"
                           )

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: Transforming Possibility into Reality**

Hope AI is a groundbreaking innovation that aims to enhance human potential through artificial intelligence. By blending machine learning with empathy, it empowers individuals and communities to overcome challenges, find solutions, and foster collaboration. Hope AI tailors resources based on users' unique needs, ensuring personalized support and guidance. In a world often weighed down by uncertainty, this technology ignites optimism, transforming aspirations into tangible outcomes. With Hope AI, we stand on the brink of a future where every dream has the potential to become reality. Embrace the power of possibility today!

-----------------------------------------------------------------

In [18]:
SEO_reviewer.chat_messages_for_summary(critic)[-1]

{'content': 'As an SEO reviewer, here are my suggestions for optimizing your content:\n\n- **Incorporate Target Keywords**: Identify and integrate relevant keywords related to "artificial intelligence," "machine learning," and "personalized solutions" to improve search visibility. Use variations naturally throughout the text, including in headings and subheadings.\n\n- **Engaging Meta Description**: Craft a compelling meta description (155-160 characters) that summarizes the content while including target keywords. This will enhance click-through rates from search results.\n\n- **Add Internal and External Links**: Link to credible external resources and related internal pages to enhance authority and provide readers with additional context, helping to boost SEO ranking and user engagement.',
 'role': 'assistant',
 'name': 'SEO_Reviewer'}

In [19]:
writer.chat_messages_for_summary(critic)

[{'content': '\n        Write a concise but engaging blogpost about\n      Hope AI . Make sure the blogpost is\n       within 100 words.\n       ',
  'role': 'user',
  'name': 'Critic'},
 {'content': "**Hope AI: Transforming Possibility into Reality**\n\nHope AI is a groundbreaking innovation that aims to enhance human potential through artificial intelligence. By blending machine learning with empathy, it empowers individuals and communities to overcome challenges, find solutions, and foster collaboration. Hope AI tailors resources based on users' unique needs, ensuring personalized support and guidance. In a world often weighed down by uncertainty, this technology ignites optimism, transforming aspirations into tangible outcomes. With Hope AI, we stand on the brink of a future where every dream has the potential to become reality. Embrace the power of possibility today!",
  'role': 'assistant',
  'name': 'Writer'},
 {'content': '### Aggregated Feedback from Reviewers:\n\n#### SEO Rev

In [20]:
legal_reviewer.chat_messages_for_summary(critic)

[{'content': 'Review the following content:\n\n **Hope AI: Transforming Possibility into Reality**\n\nHope AI is a groundbreaking innovation that aims to enhance human potential through artificial intelligence. By blending machine learning with empathy, it empowers individuals and communities to overcome challenges, find solutions, and foster collaboration. Hope AI tailors resources based on users\' unique needs, ensuring personalized support and guidance. In a world often weighed down by uncertainty, this technology ignites optimism, transforming aspirations into tangible outcomes. With Hope AI, we stand on the brink of a future where every dream has the potential to become reality. Embrace the power of possibility today!\nContext: \n```json\n{\n  "Reviewer": "SEO_Reviewer",\n  "Review": "As an SEO reviewer, here are my suggestions for optimizing your content: Incorporate Target Keywords: Identify and integrate relevant keywords related to \'artificial intelligence,\' \'machine learni

In [21]:
res

ChatResult(chat_id=78938333825899916094895883893645819163, chat_history=[{'content': '\n        Write a concise but engaging blogpost about\n      Hope AI . Make sure the blogpost is\n       within 100 words.\n       ', 'role': 'assistant', 'name': 'Critic'}, {'content': "**Hope AI: Transforming Possibility into Reality**\n\nHope AI is a groundbreaking innovation that aims to enhance human potential through artificial intelligence. By blending machine learning with empathy, it empowers individuals and communities to overcome challenges, find solutions, and foster collaboration. Hope AI tailors resources based on users' unique needs, ensuring personalized support and guidance. In a world often weighed down by uncertainty, this technology ignites optimism, transforming aspirations into tangible outcomes. With Hope AI, we stand on the brink of a future where every dream has the potential to become reality. Embrace the power of possibility today!", 'role': 'user', 'name': 'Writer'}, {'cont